In [2]:
!pip install np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 571 kB/s 
  Created wheel for np-utils: filename=np_utils-0.6.0-py3-none-any.whl size=56459 sha256=50393ef079fde7b06a8f3b24990b5ebce1dcc5c6e931b7f0d1818ecbb9c9b537
  Stored in directory: /root/.cache/pip/wheels/d2/83/71/a781667865955ae7dc18e5a4038401deb56d96eb85d3a5f1c0
Successfully built np-utils


In [ ]:
!pip install gcsfs

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import re
import np_utils
import seaborn as sns
import matplotlib.pyplot as plt
import string, os
import nltk
import re
import keras
import random
import io
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#from tensorflow.keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore")
import pickle
import gcsfs

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

with fs.open('big-data-lyrician/filtered_track_df.csv') as f:
fs = gcsfs.GCSFileSystem()    df = pd.read_csv(f)

df

In [6]:
df.head()

,artists_id,acousticness,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,...,speechiness,tempo,time_signature,track_href,uri,valence,release_date,artists_name,genres,release_year
0,68WwJXWrpo1yVOOIZjLSeT,0.0268,0.506,1.0,248777.0,0.741,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.000027,1.0,...,0.0349,94.042,4.0,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.236,2018-09-28,Evalyn,"['electropop', 'indie electro-pop', 'indie pop...",2018
1,6pSsE5y0uJMwYj83KrPyf9,0.1330,0.629,1.0,207396.0,0.706,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.000000,1.0,...,0.4360,81.220,4.0,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.543,2019-01-25,Dawn Richard,"['alternative r&b', 'deep pop r&b', 'escape ro...",2019
2,09hVIj6vWgoCDtT03h8ZCa,0.0316,0.727,1.0,218773.0,0.380,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.000000,7.0,...,0.3350,92.050,4.0,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.455,1991-09-24,A Tribe Called Quest,"['alternative hip hop', 'conscious hip hop', '...",1991
3,09hVIj6vWgoCDtT03h8ZCa,0.1140,0.887,1.0,217467.0,0.419,https://api.spotify.com/v1/tracks/6GAemDMWAXa8...,6GAemDMWAXa8wZOBkO1gar,0.268000,10.0,...,0.3530,96.149,4.0,https://api.spotify.com/v1/tracks/6GAemDMWAXa8...,6GAemDMWAXa8wZOBkO1gar,0.576,1991-09-24,A Tribe Called Quest,"['alternative hip hop', 'conscious hip hop', '...",1991
4,09hVIj6vWgoCDtT03h8ZCa,0.1980,0.674,1.0,260000.0,0.582,https://api.spotify.com/v1/tracks/5id38dZomKgo...,5id38dZomKgo2h4ccxTyzT,0.000400,0.0,...,0.2300,92.650,4.0,https://api.spotify.com/v1/tracks/5id38dZomKgo...,5id38dZomKgo2h4ccxTyzT,0.690,1992-06-01,A Tribe Called Quest,"['alternative hip hop', 'conscious hip hop', '...",1992


In [7]:
df.shape

(3486, 28)

In [8]:
df.columns

Index(['artists_id', 'acousticness', 'danceability', 'disc_number',
       'duration_ms', 'energy', 'href', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'lyrics', 'mode', 'name', 'playlist',
       'popularity', 'preview_url', 'speechiness', 'tempo', 'time_signature',
       'track_href', 'uri', 'valence', 'release_date', 'artists_name',
       'genres', 'release_year'],
      dtype='object')

In [9]:
len(pd.unique(df["genres"]))

975

In [10]:
def load_data():
    df = pd.read_csv("filtered_track_df.csv")
    df['genres'] = df.genres.apply(lambda x: [i[1:-1] for i in str(x)[1:-1].split(", ")])
    exploded_track_df = df.explode("genres")
    return exploded_track_df

In [11]:
exploded_track_df = load_data()

In [12]:
genre_names = ['dance pop', 'electronic', 'electropop', 'hip hop', 'jazz', 'k-pop', 'latin', 'pop', 'pop rap', 'r&b', 'rock']
exploded_track_df = exploded_track_df.explode("genres")[exploded_track_df.explode("genres")["genres"].isin(genre_names)]

In [13]:
pd.unique(exploded_track_df['genres'])

array(['electropop', 'r&b', 'hip hop', 'pop rap', 'rock', 'dance pop',
       'pop', 'latin', 'k-pop', 'electronic', 'jazz'], dtype=object)

In [14]:
exploded_track_df.groupby(['genres'])['id'].count()

genres
dance pop     1136
electronic     116
electropop     538
hip hop        336
jazz            13
k-pop          117
latin           48
pop           1518
pop rap        574
r&b            695
rock           845
Name: id, dtype: int64

In [15]:
for genre in genre_names:
  print(genre)
  genre_track = exploded_track_df[exploded_track_df["genres"] == genre]
  #elec = exploded_track_df[exploded_track_df["genres"] == "electronic"]
#jazz["lyrics"] = jazz.lyrics.apply(lambda x: str(x).split("."))

  genre_track = genre_track[["id","lyrics"]].drop(genre_track[genre_track.lyrics == "-99"].index)

  #genre_track.to_csv("{0}_lyrics.csv".format(genre), sep='\t', encoding='utf-8')

#Lining up all the lyrics to create corpus
  Corpus =''
  for listitem in genre_track.lyrics:
    Corpus += listitem
    
  Corpus = Corpus.lower() #converting all alphabets to lowecase 
  print("Number of unique characters:", len(set(Corpus)))

#Keeping only a limited set of characters. 
  to_remove = ['{', '}', '~', '©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ü', 'ŏ',
             'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '‚', '“', '”', 
             '…', '\u205f', '\ufeff', '!', '&', '(', ')', '*', '-',  '/', ]
  for symbol in to_remove:
    Corpus = Corpus.replace(symbol," ")

# Storing all the unique characters present in my corpus to bult a mapping dic. 
  symb = sorted(list(set(Corpus)))

  L_corpus = len(Corpus) #length of corpus
  L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
  mapping = dict((c, i) for i, c in enumerate(symb))
  reverse_mapping = dict((i, c) for i, c in enumerate(symb))

  print("Total number of characters:", L_corpus)
  print("Number of unique characters:", L_symb)

#Splitting the Corpus in equal length of strings and output target
  length = 40
  features = []
  targets = []
  for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
  L_datapoints = len(targets)
  print("Total number of sequences in the Corpus:", L_datapoints)

  f = open("text_{0}.txt".format(genre), "a")
  f.write(Corpus)
  f.close()

dance pop
Number of unique characters: 555
Total number of characters: 2132413
Number of unique characters: 525
Total number of sequences in the Corpus: 2132373
electronic
Number of unique characters: 58
Total number of characters: 150925
Number of unique characters: 45
Total number of sequences in the Corpus: 150885
electropop
Number of unique characters: 135
Total number of characters: 796487
Number of unique characters: 112
Total number of sequences in the Corpus: 796447
hip hop
Number of unique characters: 210
Total number of characters: 909431
Number of unique characters: 177
Total number of sequences in the Corpus: 909391
jazz
Number of unique characters: 40
Total number of characters: 13019
Number of unique characters: 35
Total number of sequences in the Corpus: 12979
k-pop
Number of unique characters: 1107
Total number of characters: 374767
Number of unique characters: 1093
Total number of sequences in the Corpus: 374727
latin
Number of unique characters: 140
Total number of ch